In [ ]:
def generate_instance(num_rows_columns, I, J, seed=42):
    """
    Generate I demand points and J candidate facility sites from a grid of size num_rows_columns x num_rows_columns.
    Returns: (demand_points, candidate_sites)
    """
    if seed is not None:
        random.seed(seed)
    
    # すべての格子点を生成（1始まり）
    all_points = [(x, y) for x in range(1, num_rows_columns + 1)
                         for y in range(1, num_rows_columns + 1)]
    
    # ランダムにシャッフル
    random.shuffle(all_points)
    
    # 十分な点があるか確認
    assert I + J <= len(all_points), "Grid is too small for given I and J."
    
    demand_points = all_points[:I]
    candidate_sites = all_points[I:I+J]
    
    return demand_points, candidate_sites


In [ ]:
def compute_Ui_L(wij_matrix, J_L):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    D, _ = wij_matrix.shape  # D: 需要点の数, J: 候補施設の数

    # J_L が空なら影響はゼロ
    if not J_L:
        return np.zeros(D)

    # 各需要点 i に対して、リーダーの施設 j ∈ J_L からの重みを合計する
    utility_vector = np.zeros(D)
    for j in J_L:
        # 列 j は、施設 j が各需要点に与える重み
        utility_vector += wij_matrix[:, j]

    return utility_vector


def compute_Ui_F(wij_matrix, J_F):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    D, _ = wij_matrix.shape  # D: 需要点の数, J: 候補施設の数

    # J_L が空なら影響はゼロ
    if not J_F:
        return np.zeros(D)

    # 各需要点 i に対して、リーダーの施設 j ∈ J_L からの重みを合計する
    utility_vector = np.zeros(D)
    for j in J_F:
        # 列 j は、施設 j が各需要点に与える重み
        utility_vector += wij_matrix[:, j]

    return utility_vector

In [ ]:
def compute_distances(demand_points, candidate_sites):
    D, J = len(demand_points), len(candidate_sites)  # ここで D, J を定義
    distances = np.zeros((D, J))
    for d in range(D):
        for j in range(J):
            distances[d, j] = np.sqrt(
                (demand_points[d][0] - candidate_sites[j][0]) ** 2
                + (demand_points[d][1] - candidate_sites[j][1]) ** 2
            )
    return distances

In [ ]:
def compute_wij_matrix(distances, alpha=0, beta=0.1):
    wij_matrix = np.exp(alpha - beta * distances)
    return wij_matrix

In [ ]:
def round_to_binary_best_k(v, k=2):
    """
    v : 1-D ndarray, 連続値ベクトル (0-1 区間)
    k : 1 を立てる最大個数
    ---------------------------
    上位 k 個の成分を 1 にして残りを 0 に丸める。
    """
    bin_v = np.zeros_like(v)
    if k > 0:
        top_idx = np.argsort(-v)[:k]     # 大きい順に k 個取り出す
        bin_v[top_idx] = 1.0
    return bin_v

## O-EDA

In [ ]:
def project_x(v, p):
    v = np.clip(v, 0, 1)
    if v.sum() <= p:              # already inside simplex-cap
        return v
    # Euclidean projection onto {v>=0, Σv=p}
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - p
    rho = np.nonzero(u - cssv / (np.arange(len(v)) + 1) > 0)[0][-1]
    theta = cssv[rho] / (rho + 1)
    return np.maximum(v - theta, 0)

In [ ]:
def project_y(v, r, x):
    v = np.clip(v, 0, 1 - x)      # y_j ≤ 1 - x_j
    if v.sum() <= r:
        return v
    # again project onto capped simplex
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - r
    rho = np.nonzero(u - cssv / (np.arange(len(v)) + 1) > 0)[0][-1]
    theta = cssv[rho] / (rho + 1)
    return np.maximum(v - theta, 0)

In [ ]:
def compute_Ai(x, y, w_ij, Ui_L):
    # w_ij shape: (I, J)
    quad_term = -y * x**2 + (1 + y) * x
    return Ui_L + (w_ij @ quad_term)

def compute_Bi(x, y, w_ij, Ui_L, Ui_F):
    lin_term = (1 - y) * x + y
    return Ui_L + Ui_F + (w_ij @ lin_term)

In [ ]:
def grad_A_x(x, y, w_ij):         # shape (I,J)
    return w_ij * (-2 * y * x + (1 + y))

def grad_A_y(x, y, w_ij):
    return w_ij * (x - x**2)

def grad_B_x(x, y, w_ij):
    return w_ij * (1 - y)

def grad_B_y(x, y, w_ij):
    return w_ij * (1 - x)

In [ ]:
def ogda_dinkelbach(w_ij, Ui_L, Ui_F, h_i, p, r,
                    eta=5e-2, max_iter=500, tol=1e-6, seed=0):
    
    hist_Lhat,hist_Lcont, hist_dx, hist_dy = [], [], [], []

    rng = np.random.default_rng(seed)
    J = w_ij.shape[1]
    I = w_ij.shape[0]

    # init feasible
    x = project_x(rng.random(J), p)
    y = project_y(np.zeros(J), r, x)
    gamma = np.zeros(I)

    g_prev_x = np.zeros_like(x)
    g_prev_y = np.zeros_like(y)

    for t in range(max_iter):
        print(f"\n🌸🌸🌸  Iteration {t+1}  🌸🌸🌸")

        # --- Ai, Bi ---
        Ai = compute_Ai(x, y, w_ij, Ui_L)
        Bi = compute_Bi(x, y, w_ij, Ui_L, Ui_F)

        # --- γ update (Dinkelbach) ---
        gamma = Ai / Bi

        # --- gradients of Φ ---
        #   ∇Φ = Σ_i h_i (∇A_i - γ_i ∇B_i)
        grad_x = (h_i[:, None] * (grad_A_x(x, y, w_ij) -
                                  gamma[:, None] * grad_B_x(x, y, w_ij))).sum(axis=0)
        grad_y = (h_i[:, None] * (grad_A_y(x, y, w_ij) -
                                  gamma[:, None] * grad_B_y(x, y, w_ij))).sum(axis=0)

        # --- Optimistic update ---
        x_new = project_x(x + eta * (grad_x - g_prev_x), p)
        y_new = project_y(y - eta * (grad_y - g_prev_y), r, x_new)

        # --- 履歴 ---
        x_bin, y_bin = round_disjoint_best_k(x_new, y_new, p, r)
        x_bin_bin = round_to_binary_best_k(x_new, p)
        y_bin_bin = round_to_binary_best_k(y_new, r)
        Lval = compute_L(h_i, Ui_L, Ui_F, w_ij, x_bin, y_bin)
        Lhat = calc_Lhat(x, y, w_ij, Ui_L, Ui_F, h_i)

        hist_Lhat.append(Lhat)
        hist_Lcont.append(Lval)
        hist_dx.append(np.linalg.norm(x_new - x))
        hist_dy.append(np.linalg.norm(y_new - y))

        print("🔧  After rounding:")
        print(f"     ➤ x (rounded): {x_bin}")
        print(f"     ➤ x ( binbin): {x_bin_bin}")
        print(f"     ➤ y (rounded): {y_bin}")
        print(f"     ➤ y ( binbin): {y_bin_bin}")
        print(f"📈  Objective L(x, y) = {Lval:.6f}")
        print(f"📈  Objective L̂(x, y) = {Lval:.6f}")
        print(f"🔍  dx = {hist_dx[-1]:.2e}, dy = {hist_dy[-1]:.2e}")

        # --- check convergence (Φ close to 0) ---
        phi_val = (h_i * (Ai - gamma * Bi)).sum()
        if abs(phi_val) < tol and max(np.linalg.norm(x_new - x),
                                      np.linalg.norm(y_new - y)) < tol:
            print("🎉✨ 収束しました！Great job! ✨🎉")
            print(f"Converged at iter {t}, Φ={phi_val:.2e}")
            break

        # shift for next optimistic step
        g_prev_x, g_prev_y = grad_x, grad_y
        x, y = x_new, y_new

    return x, y, gamma, hist_Lhat, hist_Lcont, hist_dx, hist_dy


In [ ]:
def compute_L(h_i, Ui_L, Ui_F, wij, x, y):
    """
    関数 L(x, y) を計算する

    Parameters:
        h (np.array): 需要点ごとの人口密度ベクトル (D,)
        Ui_L (np.array): 各需要点におけるリーダーの影響度 (D,)
        Ui_F (np.array): 各需要点におけるフォロワーの影響度 (D,)
        wij (np.array): 需要点と施設候補の重み行列 (D, J)
        x (np.array): リーダーが選択した施設配置 (J,)
        y (np.array): フォロワーが選択した施設配置 (J,)

    Returns:
        float: L(x, y) の計算結果
    """
    numerator = Ui_L + (wij @ x)  # 分子: リーダーの影響度 + 選択した施設の影響
    denominator = Ui_L + Ui_F + (wij @ np.maximum(x, y))  # 分母: 総合影響度

    return np.sum(h_i * (numerator / denominator))

In [ ]:
def round_to_binary_best_k(v, k):
    """
    v : 1-D ndarray, 連続値ベクトル (0-1 区間)
    k : 1 を立てる最大個数
    ---------------------------
    上位 k 個の成分を 1 にして残りを 0 に丸める。
    """
    bin_v = np.zeros_like(v)
    if k > 0:
        top_idx = np.argsort(-v)[:k]     # 大きい順に k 個取り出す
        bin_v[top_idx] = 1.0
    return bin_v

In [ ]:
def round_disjoint_best_k(x_cont, y_cont, p, r):
    """
    x 用の連続値スコア x_cont,  y 用の連続値スコア y_cont を
    0-1 ベクトル (x_bin, y_bin) に丸める。
    - x は上位 p 個を 1
    - y は x が 1 でない場所から上位 r 個を 1
    """
    J = len(x_cont)
    x_bin = np.zeros(J)
    y_bin = np.zeros(J)

    # --- x を確定 ---
    idx_x = np.argsort(-x_cont)[:p]
    x_bin[idx_x] = 1

    # --- y を確定 (x が 0 の場所限定) ---
    candidate_mask = (x_bin == 0)
    idx_y_all = np.argsort(-y_cont)           # 降順ソート
    # フィルタして上位 r 個
    selected_y = [j for j in idx_y_all if candidate_mask[j]][:r]
    y_bin[selected_y] = 1

    return x_bin, y_bin

In [ ]:
def calc_Lhat(x: np.ndarray,          # shape (J,)
              y: np.ndarray,          # shape (J,)
              w_ij: np.ndarray,       # shape (I, J)
              Ui_L: np.ndarray,       # shape (I,)
              Ui_F: np.ndarray,       # shape (I,)
              h_i : np.ndarray):      # shape (I,)
    """
    Compute  \hat{L}(x, y)  defined by
        sum_i h_i * (Ui_L + Σ_j w_ij * [-y_j x_j^2 + (1+y_j)x_j]) /
                        (Ui_L + Ui_F + Σ_j w_ij * [(1-y_j)x_j + y_j])
    """
    # ----- numerator:  Ui_L + Σ_j w_ij · (-y x^2 + (1+y)x) -----
    quad_term = -y * x**2 + (1 + y) * x              # shape (J,)
    Ai = Ui_L + (w_ij @ quad_term)                   # shape (I,)

    # ----- denominator: Ui_L + Ui_F + Σ_j w_ij · ((1-y)x + y) -----
    lin_term  = (1 - y) * x + y                      # shape (J,)
    Bi = Ui_L + Ui_F + (w_ij @ lin_term)             # shape (I,)

    # ----- weighted sum -----
    return np.sum(h_i * Ai / Bi)
